# Poisson Variational Autoencoder (P-VAE)

This notebook provides a concise, self-contained implementation of a Poisson Variational Autoencoder. Our goal is to present a clear and educational example that you can learn from directly. For more extensive experiments, detailed configurations, and full reproducibility, please refer to the complete codebase associated with this project.

This notebook uses pytorch lightning and wandb for clarity and ease of monitoring runs. The code can easily be adapted to avoid using using these tools by commenting out logging calls and writing your own training loop.

In [1]:
import torch.nn as nn
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision, math, os, torch
import lightning as pl
import wandb as wdb
from lightning.pytorch.loggers import wandb
name = 'PVAE_minimal' #change to your run name
data_dir = "/Data/Datasets/" #change to your data directory
project_name = "PVAE" #change to your wandb project name
root_dir = "data"
bsize = 256
train_device = "0"
device = train_device + "," #lightning device formatting
checkpoint_dir = os.path.join(root_dir, name)
os.makedirs(checkpoint_dir, exist_ok=True)

In [2]:
ds_train = datasets.MNIST(data_dir, train=True, download=True).data
ds_test = datasets.MNIST(data_dir, train=False, download=True).data
ds_train = (ds_train.float().reshape(-1, 784)/255)
ds_test = (ds_test.float().reshape(-1, 784)/255)
ds_train = torch.utils.data.TensorDataset(ds_train)
ds_test = torch.utils.data.TensorDataset(ds_test)

train_dl = DataLoader(ds_train, batch_size=bsize, shuffle=True)
val_dl = DataLoader(ds_test, batch_size=bsize, shuffle=False)

First lets define a Poisson distribution object that allows to differentiate through the sampling process with rsample. This object also provides a function for computing the kl from the prior distribution.

In [3]:
class Poisson:
    def __init__(self, log_rate, t=0.0):
        self.log_rate = log_rate
        self.rate = torch.exp(
            self.log_rate.clamp(None, 5)
        ) + 1e-6
        self.n_trials = int(math.ceil(max(self.rate.max().item(),1)*5)) # a large enough number of trials to sample from
        self.t = t
    def rsample(self, hard: bool = False):
        x = torch.distributions.Exponential(self.rate).rsample((self.n_trials,))  # inter-event times
        times = torch.cumsum(x, dim=0)  # arrival times of events
        indicator = times < 1.0 # did events arrive before the end of the time interval
        if not (hard or self.t == 0): # soften the indicator function
            indicator = torch.sigmoid((1.0 - times) / self.t)
        return indicator.sum(0).float()
    def kl(self, prior, du):
        #"prior" argument referes to log rate of prior
        #equation is r * (1 - dr + dr * log(dr))
        r = torch.exp(prior.clamp(None, 5)) + 1e-6
        rdr = self.rate #final rate is rdr
        logdr = du #log of the modulation of prior rate
        return r-rdr+rdr*logdr

This example pytorch module defines the encoder, decoder, and forward (autoencoding) process of the VAE.

In [4]:
class PVAE(nn.Module):
    def __init__(self):
        super(PVAE, self).__init__()
        self.encode = nn.Sequential(
            nn.Linear(784, 128),
        )
        self.decode = nn.Sequential(
            nn.Linear(128, 784),
            nn.Sigmoid(),
        )
        self.prior = nn.Parameter(torch.zeros((1, 128)))
        self.t = 1.0 #temperature
    def forward(self, x):
        validation = not torch.is_grad_enabled()
        du = self.encode(x).clamp(None, 5)
        dist = Poisson((du + self.prior.clamp(None, 5)).clamp(None, 5), self.t)
        z = dist.rsample(hard=validation)
        y = self.decode(z)
        return dist, du, z, y

We now define the Lightning Module for training, including the training and validation steps, the optimizer, and beta/temperature scheduling. Within these lines we also log to the wandb interface.

In [5]:
class PL_PVAE(pl.LightningModule):
    def __init__(self):
        super(PL_PVAE, self).__init__()
        self.model = PVAE()
        self.opt = torch.optim.Adam
        self.opt_params = {
            'lr': 1e-3,
        }
        self.beta = 0.0
    def forward(self, x):
        return self.model(x[0].flatten(1))
    def training_step(self, batch, batch_idx):
        epoch = self.current_epoch + batch_idx/len(train_dl)
        self.beta = min(5.0, 5*epoch/250)
        self.model.t = max((1.0 - 0.95*epoch/250), 0.05)
        self.log('beta', self.beta)
        self.log('t', self.model.t)
        
        dist, du, z, y = self(batch)
        kl = dist.kl(self.model.prior, du).mean()
        mse = ((y - batch[0])**2).sum(-1).mean()
        loss = self.beta*kl + mse
        self.log('train_loss', loss.item(), on_step=True, on_epoch=True, prog_bar=True)
        self.log('train_elbo', (kl + mse).item(), on_step=True, on_epoch=True, prog_bar=True)
        return loss
    @torch.no_grad()
    def validation_step(self, batch, batch_idx):
        dist, du, z, y = self(batch)
        kl = dist.kl(self.model.prior, du).mean()
        mse = ((y - batch[0])**2).sum(-1).mean()
        loss = self.beta*kl + mse
        
        self.log('val_mse', mse.item(), on_step=True, on_epoch=True, prog_bar=True)
        self.log('val_kl', kl.item(), on_step=True, on_epoch=True, prog_bar=True)
        self.log('val_elbo', (kl + mse).item(), on_step=True, on_epoch=True, prog_bar=True)
        self.log('l0_sparsity', (z == 0).float().mean().item(), on_step=True, on_epoch=True, prog_bar=True)
        if batch_idx == 0:
            fig_y = torchvision.utils.make_grid(y.reshape(-1, 1, 28, 28), nrow=10)
            fig_x = torchvision.utils.make_grid(batch[0].reshape(-1, 1, 28, 28), nrow=10)
            self.logger.experiment.log({
                'recons': wdb.Image(fig_y, caption="recons"),
                'inputs': wdb.Image(fig_x, caption="inputs"),
            })
        return loss
    def configure_optimizers(self):
        return self.opt(self.parameters(), **self.opt_params)

The following code starts the training process. One thing to note, is that for training we use a soft version of Poisson to get gradients, and for validation we use standard poisson. For this reason, training loss will tend to be better than validation loss particularly at the beginning of training. Training loss can also increase during training because of the temperature scheduling.

In [6]:
model = PL_PVAE()
trainer_args = {
    "callbacks": [pl.pytorch.callbacks.ModelCheckpoint(dirpath=checkpoint_dir, monitor='val_elbo', save_top_k=1, mode='min', verbose=True)],
    "accelerator": "cpu" if device == 'cpu' else "gpu",
    "logger": wandb.WandbLogger(project=project_name, name=name, save_code=True),
    "gradient_clip_val": 1.0,
}
trainer_args["logger"].watch(model, log="all")
if device != 'cpu':
    trainer_args["devices"] = device
trainer = pl.Trainer(**trainer_args, default_root_dir=checkpoint_dir, max_epochs=500, num_sanity_val_steps=0)
trainer.fit(model, val_dataloaders=val_dl, train_dataloaders=train_dl)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: galor (yateslab). Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/dekel/.conda/envs/v11/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/dekel/Documents/VAE/data/PVAE_minimal exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type | Params | Mode 
---------------------------------------
0 | model | PVAE | 201 K  | train
---------------------------------------
201 K     Trainable params
0         Non-trainable params
201 K   

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 235: 'val_elbo' reached 22.09464 (best 22.09464), saving model to '/home/dekel/Documents/VAE/data/PVAE_minimal/epoch=0-step=235.ckpt' as top 1


...

Epoch 497, global step 117030: 'val_elbo' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 498, global step 117265: 'val_elbo' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 499, global step 117500: 'val_elbo' was not in top 1
`Trainer.fit` stopped: `max_epochs=500` reached.
